<a href="https://colab.research.google.com/github/MusaibBashir/ml/blob/main/Himanshu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df=pd.read_csv('train_dataset.csv')
df.head()

,departure_station,arrival_station,departure_time,arrival_time,day_of_week,train_type,distance_km,train_number
0,Atlanta,Washington DC,09:30,10:11,1,Regional,63,3
1,Los Angeles,Boston,08:00,09:15,3,Regional,101,10
2,Washington DC,New York,07:30,09:21,1,Local,88,2
3,New York,Chicago,08:30,10:38,2,Local,97,4
4,Los Angeles,Atlanta,14:45,16:20,1,Local,74,9


In [3]:
df.size

32000

In [4]:
def time_to_minutes(t):
    h, m = map(int, t.split(":"))
    return h * 60 + m

df['departure_time'] = df['departure_time'].apply(time_to_minutes)
df['arrival_time'] = df['arrival_time'].apply(time_to_minutes)

In [5]:
df['duration_minutes'] = df['arrival_time'] - df['departure_time']
df['speed'] = df['distance_km'] / (df['duration_minutes'] / 60)

In [7]:
df.head()

,departure_station,arrival_station,departure_time,arrival_time,day_of_week,train_type,distance_km,train_number,duration_minutes,speed
0,Atlanta,Washington DC,570,611,1,Regional,63,3,41,92.195122
1,Los Angeles,Boston,480,555,3,Regional,101,10,75,80.800000
2,Washington DC,New York,450,561,1,Local,88,2,111,47.567568
3,New York,Chicago,510,638,2,Local,97,4,128,45.468750
4,Los Angeles,Atlanta,885,980,1,Local,74,9,95,46.736842


In [8]:
categorical_cols = ['day_of_week', 'train_type','speed']
df_encoded = pd.get_dummies(df, columns=categorical_cols)

In [9]:
X = df_encoded.drop(['departure_station','arrival_station','train_number'], axis=1)
y = df['train_number']

In [10]:
df.head()

,departure_station,arrival_station,departure_time,arrival_time,day_of_week,train_type,distance_km,train_number,duration_minutes,speed
0,Atlanta,Washington DC,570,611,1,Regional,63,3,41,92.195122
1,Los Angeles,Boston,480,555,3,Regional,101,10,75,80.800000
2,Washington DC,New York,450,561,1,Local,88,2,111,47.567568
3,New York,Chicago,510,638,2,Local,97,4,128,45.468750
4,Los Angeles,Atlanta,885,980,1,Local,74,9,95,46.736842


In [11]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [14]:
model = Sequential([
    Dense(10, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(10, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1208 - loss: 2.6539 - val_accuracy: 0.0922 - val_loss: 2.5797
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1303 - loss: 2.3265 - val_accuracy: 0.0922 - val_loss: 2.5432
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1830 - loss: 2.2363 - val_accuracy: 0.0875 - val_loss: 2.5204
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2016 - loss: 2.1500 - val_accuracy: 0.0875 - val_loss: 2.5249
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2164 - loss: 2.1017 - val_accuracy: 0.0938 - val_loss: 2.5404
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2297 - loss: 2.0667 - val_accuracy: 0.1000 - val_loss: 2.5735
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2559 - loss: 2.0285 - val_accuracy: 0.0953 - val_loss: 2.6149
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2602 - loss: 1.9911 - val_accuracy: 0.1016 - v

In [16]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0669 - loss: 6.6152     
Test Accuracy: 0.08
